In [0]:
import zipfile
with zipfile.ZipFile('./house-prices-advanced-regression-techniques.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [0]:
import pandas as pd
import numpy as np

In [0]:
train_data=pd.read_csv('./train.csv')
test_data=pd.read_csv('./test.csv')

In [0]:
train_data.head(5)

In [37]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [0]:
train_data.info()

In [0]:
test_data.info()

In [0]:
## Top 20 Fields of train data having null values
train_data.isnull().sum().sort_values(ascending=False).iloc[:20]

In [0]:
## Top 35 Fields of test data having null values
test_data.isnull().sum().sort_values(ascending=False).iloc[:35]

In [0]:
len(train_data),len(test_data)

In [0]:
train_data.columns

In [0]:
test_data['Alley'].isnull().sum()

1352

In [0]:
def filter(dataset):
    n=len(dataset)
    ll=[]
    for col in dataset.columns:
        if( dataset[col].isnull().sum()>=int(n*0.2) ):
            ll.append(col)
    print(ll,"features removed from provided data")
    dataset.drop(ll,axis=1,inplace=True)
    return dataset

In [72]:
train_data = filter(train_data)
test_data = filter(test_data)

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'] features removed from provided data
['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'] features removed from provided data


In [0]:
train_data.isnull().sum().sort_values(ascending=False).iloc[:20]

In [0]:
## Top 35 Fields of test data having null values
test_data.isnull().sum().sort_values(ascending=False).iloc[:35]

In [0]:
train_data.info()

In [0]:
test_data.info()

In [0]:
## Ignore this cell it was for experimental purpose

from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=7) 
  
knn.fit(X_train, y_train) 
  
# Predict on dataset which model has not seen before 
print(knn.predict(X_test))

In [0]:
def filter1(dataset):
    for col in dataset.columns:
        #print(dataset[col].dtypes)
        if dataset[col].dtypes=='object':
            dataset[col].fillna(dataset[col].mode()[0],inplace=True)
        elif dataset[col].dtypes=='int64' or dataset[col].dtypes=='float64':
            dataset[col].fillna(dataset[col].mean(),inplace=True)            

    return dataset

In [0]:
train_data=filter1(train_data)

In [0]:
test_data=filter1(test_data)

In [0]:
## Top 5 Fields of test data having null values
test_data.isnull().sum().sort_values(ascending=False).iloc[:5]

In [0]:
## Top 5 Fields of test data having null values
train_data.isnull().sum().sort_values(ascending=False).iloc[:5]

In [0]:
train_data_len = len(train_data)
test_data_len = len(test_data)

In [0]:
print(train_data.shape,test_data.shape)

(1460, 76) (1459, 75)


In [76]:
total = pd.concat([train_data.drop(['SalePrice'],axis=1),test_data],axis=0)
total.shape

(2919, 75)

In [0]:
def category_onehot_multcols(multcolumns):
    df_final=total
    i=0
    for fields in multcolumns:
        
        print(fields)
        df1=pd.get_dummies(total[fields],drop_first=True)
        
        total.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:
            
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1
       
        
    df_final=pd.concat([total,df_final],axis=1)
        
    return df_final

In [0]:
column=['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood',
         'Condition2','BldgType','Condition1','HouseStyle','SaleType',
        'SaleCondition','ExterCond',
         'ExterQual','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
        'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Heating','HeatingQC',
         'CentralAir',
         'Electrical','KitchenQual','Functional',
         'GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive']

In [0]:
total=category_onehot_multcols(column)

In [80]:
from sklearn.preprocessing import StandardScaler
x = total.values
x = StandardScaler().fit_transform(x)

x.shape

(2919, 233)

In [0]:
## naming columns to ease the operations

cols=[]
for i in range(0,233):
    name = "col"+str(i)
    cols.append(str(name))

cols
x=pd.DataFrame(x,columns=cols)

In [0]:
train = x.iloc[:1460]
test = x.iloc[1460:]

In [0]:
y = train_data['SalePrice'].values

sc=StandardScaler()

y = pd.DataFrame(sc.fit_transform(y.reshape(-1,1)))

y.columns=['SalePrice']

train = pd.concat([train,y],axis=1)

In [192]:
import seaborn as sns 
import matplotlib.pyplot as plt 
from scipy.stats import norm 

interested_col=[]

for item in train.corr().reset_index()[['index','SalePrice']].values:
    if item[1:2]>=0.15 or item[1:2]<=-0.15:
        interested_col.append(item[0:1][0])

interested_col.remove('SalePrice')
print(interested_col)


['col2', 'col3', 'col4', 'col6', 'col7', 'col8', 'col9', 'col11', 'col12', 'col13', 'col14', 'col16', 'col17', 'col19', 'col20', 'col21', 'col23', 'col24', 'col25', 'col26', 'col27', 'col28', 'col29', 'col39', 'col40', 'col44', 'col61', 'col63', 'col66', 'col69', 'col70', 'col71', 'col76', 'col102', 'col110', 'col112', 'col116', 'col117', 'col123', 'col124', 'col125', 'col126', 'col131', 'col132', 'col136', 'col138', 'col140', 'col149', 'col151', 'col160', 'col168', 'col172', 'col173', 'col182', 'col187', 'col188', 'col190', 'col191', 'col192', 'col201', 'col202', 'col206', 'col207', 'col208', 'col209', 'col216', 'col218', 'col220', 'col221', 'col222', 'col232']


In [193]:
columns=interested_col
print(columns)

['col2', 'col3', 'col4', 'col6', 'col7', 'col8', 'col9', 'col11', 'col12', 'col13', 'col14', 'col16', 'col17', 'col19', 'col20', 'col21', 'col23', 'col24', 'col25', 'col26', 'col27', 'col28', 'col29', 'col39', 'col40', 'col44', 'col61', 'col63', 'col66', 'col69', 'col70', 'col71', 'col76', 'col102', 'col110', 'col112', 'col116', 'col117', 'col123', 'col124', 'col125', 'col126', 'col131', 'col132', 'col136', 'col138', 'col140', 'col149', 'col151', 'col160', 'col168', 'col172', 'col173', 'col182', 'col187', 'col188', 'col190', 'col191', 'col192', 'col201', 'col202', 'col206', 'col207', 'col208', 'col209', 'col216', 'col218', 'col220', 'col221', 'col222', 'col232']


In [0]:
## ignore this cell it was for experimental purpose but showed more error in prediction

from sklearn.decomposition import PCA

pca = PCA(n_components=20)
principalComponents = pca.fit_transform(x)

Df = pd.DataFrame(data = principalComponents, columns = ['pc1', 'pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10','pc11', 'pc12','pc13','pc14','pc15','pc16','pc17','pc18','pc19','pc20'])

print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

Explained variation per principal component: [0.05641048 0.03136899 0.02432179 0.02185024 0.02067537 0.01807903
 0.01644024 0.01634451 0.01553345 0.01450796 0.0142437  0.01336607
 0.01299529 0.01241736 0.0123091  0.01183388 0.01177419 0.01125267
 0.01109778 0.0107838 ]


In [0]:
from tensorflow.keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [121]:
## ignore this cell it was for experimental purpose but showed more error in prediction

from sklearn.ensemble import RandomForestRegressor 
  
 # create regressor object 
regressor = RandomForestRegressor(n_estimators = 500, random_state = 0) 
  
# fit the regressor with x and y data 
regressor.fit(train.drop(['SalePrice'],axis=1)[columns], y)

y_pred = regressor.predict(test[columns])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


In [197]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(data=train[columns],label=y)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 1000)

xg_reg.fit(train[columns],y)

y_pred = xg_reg.predict(test[columns])

[10:58:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [198]:
pred=pd.DataFrame(sc.inverse_transform(y_pred))
sub_df=pd.read_csv('sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred],axis=1)
datasets.columns=['Id','SalePrice']

# datasets.drop([1459],inplace=True)
datasets.isnull()
datasets.to_csv('sample_submission.csv',index=False)

datasets.head()

,Id,SalePrice
0,1461,120178.632812
1,1462,157309.953125
2,1463,184031.468750
3,1464,194357.015625
4,1465,179637.234375
